# Dataset selection

Define a rule to select the datasets among a list of 466 datasets.  
We're looking for real datasets used for classification and where adding synthetic in the training can improve the classifer.
To do this, we define the 'Sum of Model Performance' as SMP : Accuracy + Recall + AUC + Precision + F1. The SMP is defined between 0 and 5. 
We compute it for every datasets and multiple train and test set. We keep the datasets with a mean SMP, 2\<Mean SMP\<4


In [27]:
import pycaret.classification as pc
import pycaret.regression as pr
import pandas as pd
import os 
from os import path as pa
import numpy as np
import json
from tqdm.notebook import tqdm
import copy
from collections import Counter
import scipy
from preferencefig import *
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split


In [2]:
directory = '../T_Data/'                                            # Where find the datasets following notebook N1
directory_to_save = '../Results/Result_11/Data/list_datasets.csv'   # Where to save the list of selected datasets
list_data = os.listdir(directory)
ser_names = pd.Series(list_data)
num_try = 2

smp = np.zeros((len(list_data),num_try))
i = 0
for path in tqdm(list_data) : 
    for n_try in range(num_try) : 
        real_data = pd.read_csv(directory+path+'/Data/'+path+'.csv',index_col=[0])
        X_train, X_test = train_test_split(real_data, test_size=0.33)
        
        f = open(directory+'/'+path+'/'+'dict_info.json') 
        dict_info=json.load(f)
        f.close()
        l = pc.setup(X_train, 
                target =dict_info['target'],
                test_data = X_test,fold_strategy = "kfold",
                fold=20,
                numeric_features = dict_info['numeric_features'],
                categorical_features = dict_info['categorical_features'],
                ignore_features = dict_info['ignore_features'],
                normalize = True,
                normalize_method = "zscore",
                silent = True, verbose = False) 
        r1 = pc.compare_models(n_select=15)
        l = pc.pull()
        if not l.empty : 
            smp[i,n_try] = l.iloc[0][['Accuracy','AUC','Recall','Prec.','F1']].sum()
    i+=1
       
ser_names.loc[(smp.sum(axis=1)<4)&(smp.sum(axis=1)>2)].to_csv(directory_to_save)

In [18]:
ser_names.loc[(smp.sum(axis=1)>0.5)].to_csv('../Results/Result_11/Data/test_dataset_selection.csv')

In [19]:
test = pd.read_csv('../Results/Result_11/Data/test_dataset_selection.csv',index_col=[0])
print(test)

                                       0
9                         fri_c1_500_5_1
10                       fri_c2_250_50_1
13              analcatdata_bankruptcy_1
16                       mfeat-zernike_1
17                           autoPrice_1
23                     ozone-level-8hr_1
24                 arsenic-female-lung_1
27                       fri_c1_500_10_1
28                       fri_c0_500_10_1
29                             segment_1
31                        fri_c0_100_5_1
32                         mammography_1
35                         qsar-biodeg_1
36          meta_batchincremental.arff_1
38                      fri_c2_1000_25_1
39                   autoUniv-au4-2500_1
40                           PieChart2_1
45                            rabe_131_1
47                         page-blocks_1
56                         chatfield_4_1
57                       fri_c3_250_10_1
59                      cylinder-bands_2
62                       fri_c0_500_50_1
70              

In [ ]:
l.iloc[0][['Accuracy','AUC','Recall','Prec.','F1']].sum()

In [ ]:
smp[i,n_try]

In [ ]:
l.iloc[0][['Accuracy','AUC','Recall','Prec.','F1']].sum()

In [ ]:
0.83+0.62+0.02+0.69+0.04